In [12]:
import pandas as pd
import numpy as np
# Название файла
name = 2
name_files = str(name) + ".xlsx"
# name_files = "1_1.xlsx"
# Загрузка файлов
data = pd.DataFrame(pd.read_excel(name_files, header=None))
ansi_table_before25 = pd.DataFrame(pd.read_excel("./tables/ansi_ss_before25.xlsx", header=None))
ansi_table_after25 = pd.DataFrame(pd.read_excel("./tables/ansi_ss_after25.xlsx", header=None))
ansi_table_ansi_cs = pd.DataFrame(pd.read_excel("./tables/ansi_cs.xlsx", header=None))
name_of_columns = pd.DataFrame(pd.read_excel("./tables/vars_for_rows.xlsx", header=None))
vars_material = pd.DataFrame(pd.read_excel("./tables/english_vars_material.xlsx", header=None))
def Clear_Spec(): #Отчистка от всего чего не должно быть
    global data
    data = data.dropna(thresh=5) #Удаление строк.
    data = data.dropna(axis = 1, thresh = (data.shape[1] // 5)) #Удаление столбцов.
    data.reset_index(inplace = True, drop = True) #Сбрасываем индекс.
    n = 10
    flag = False
    for Y in range(data.shape[0]): 
        count = 1
        for X in range(n):
            if flag:
                break
            if count >= 5:
                data = data.drop(list(data.index.values)[Y])
                flag = True
                break
            if data.iloc[Y, X] == count:
                count += 1
    data = data.T
    data.reset_index(inplace = True, drop = True) #Сбрасываем индекс.
    data = data.T
    
def Column_search(): # Поиск нужных характеристик материала
    global data
    global result
    flag_0 = False
    flag_1 = False
    flag_2 = False
    flag_3 = False
    pointer = 0
    data_for_parsing = data
    data_for_parsing = data_for_parsing.drop(index= 0)
    data_for_parsing.reset_index(inplace = True, drop = True) #Сбрасываем индекс.
    data.reset_index(inplace = True, drop = True) #Сбрасываем индекс.
    result = pd.DataFrame(np.zeros((data_for_parsing.shape[0]-1,  ), dtype=[
    ('Наименование','a4'),
    ('Диаметр','f4'),
    ('Давление','f4'),
    ('Температура','f4'),
    ('Корпус','a4'),
    ('Присоединение','a4')
    ]))
    headers = ['Наименование','Диаметр','Давление','Температура','Корпус','Присоединение']
    # Пробег по клиентской таблице
    for Y in list(data.columns.values):
        if flag_0 != True:
            #Пробег по таблице наименований
            for B in list(name_of_columns.columns.values):
                if flag_2 != True:
                    for A in list(name_of_columns.index.values):
                        if flag_3 != True:
                            if (str(data.iloc[0,Y]).lower() == str(name_of_columns.iloc[A,B]).lower()):
                                data.to_excel("data.xlsx")
                                data_for_parsing.to_excel("data_for_parsing.xlsx")
                                for G in (data_for_parsing.index.values):
                                    data_column = data_for_parsing[Y]
                                    result[headers[B]] = data_column.iloc[0:data_column.shape[0]]
                                pointer += 1
                                if pointer == 6:
                                    flag_3 = True
                                    break
                        else:
                            flag_2 = True
                            break
                else:
                    flag_1 = True
                    break
        else:
            flag_0 = True
            break
    return(result)

def ANSI_column(result): # Создание колонки ANSI
    global vars_material
    vars_material = list(vars_material[0])
    header_ANSI = ['ANSI']
    ANSI = [[]]
    for main_x in range (0, result.shape[0]):
        stop_point = 1
        #Выбор талаблицы расчета ANSI
        ss = False
        for vars_material_key in range (0, len(vars_material)):
            if (result.iloc[(main_x, 4)]) == vars_material[vars_material_key]:
                ss = True
                if result.iloc[(main_x, 1)]  <= 25:
                    ansi_table = ansi_table_before25
                else:
                    ansi_table = ansi_table_after25
        
        if  ss == False:
            ansi_table = ansi_table_ansi_cs
        #Выбор коэффициента ANSI
        for k in range (1, ansi_table.shape[0]):
            if result.iloc[(main_x, 3)] <= ansi_table.iloc[(k,0)] and stop_point == 1:
                for j in range (1,ansi_table.shape[1]):
                    if result.iloc[(main_x, 2)] <= ansi_table.iloc[(k,j)] and stop_point == 1:
                        ANSI[0].append(ansi_table.iloc[(0,j)])
                        stop_point = 2
                        break
    error = "[[]]'"
    ANSI_list = ANSI
    for char in error:
        ANSI_list = str(ANSI_list).replace(char,"")
    ANSI_list = ANSI_list.split(',', maxsplit=-1)
    result['ANSI'] = ANSI_list
    return(result)

#вызов функций
Clear_Spec()
Column_search() 
ANSI_column(result)
name_files_w = name_files.split('.', maxsplit=1)
print(result)
result.to_excel("result/" + str(name_files_w[0]) + "writer.xlsx")

                     Наименование Диаметр Давление Температура Корпус  \
0  Клапан регулирующий сильфонный      10       20         300     нж   
1  Клапан регулирующий сильфонный      10       20         300     нж   
2  Клапан регулирующий сильфонный      10       20         300     нж   
3  Клапан регулирующий сильфонный      32       11         300     нж   
4  Клапан регулирующий сильфонный      32       11         300     нж   
5  Клапан регулирующий сильфонный      10       20         300     нж   
6  Клапан регулирующий сильфонный      10       20         300     нж   
7  Клапан регулирующий сильфонный      10       20         300     нж   
8  Клапан регулирующий сильфонный      32       11         300     нж   

  Присоединение     ANSI  
0  под приварку   2500.0  
1  под приварку   2500.0  
2  под приварку   2500.0  
3  под приварку   1500.0  
4  под приварку   1500.0  
5  под приварку   2500.0  
6  под приварку   2500.0  
7  под приварку   2500.0  
8  под приварку   1500.0  

In [18]:
vars_material

['SS',
 ' / нж',
 '08X18H10T',
 '08Х18Н10',
 '08Х18Н10Т',
 'SAF2507',
 'ss / нж',
 'ss\nнж',
 'ss/нж',
 'stainless steel / нж',
 'корпус ст.08Х18Н10Т с патрубками из ст.20',
 'Коррозионостойкая сталь',
 'нерж. сталь',
 'нерж.сталь\nstainless steel',
 'Нержавеющая сталь ',
 'нж',
 'Нж / Ss',
 'нж.',
 'нж. ст.',
 'Сборный / Composite',
 'сборный\n08Х18Н10Т',
 'сталь',
 'Сталь 08Х18Н10Т',
 'Сталь 12Х18Н10Т',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [17]:
vars_material[2]

'08X18H10T'

In [19]:

    ANSI = pd.DataFrame(np.zeros((1,  ), dtype=[('ANSI','a4')]))

IndentationError: unexpected indent (<ipython-input-19-e430bb98bbfa>, line 2)

In [169]:
data


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,38,39,40,41,42,43
0,№ п/п,Идентификатор,Маркировка арматуры,Наименование,Тип,Класс и группа безопас-ности изделия по НП-68-05,Категория ОК,Оборудование/Материалы,"DN(арматуры), мм","Pp (арматура АЭС), Pу (общепром. арматура), МПа",...,Дата выплаты авансового платежа,Размер авансового платежа (30% от стоимости Об...,"Размер авансового платежа руб., в т.ч. НДС",Объект проектирования,Примечание,Срок поставки,Разработчик РД,Код ЕОС НСИ (GID),задание,ДЕЛЕНИЕ
2,1,2.ИСУП.5872275,11LAB80AA002,Задвижка,ЗД30-500-120-250-Э/П-Н с электроприводом Э/П,2BIIIa,QA2,оборудование\r\n,200,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA),10UJE; tсраб=150 s; Inom=20.5 A. tсраб=150 s;...,2020-03-29 00:00:00,АЭП;БКП-1;Ооб,1037211,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
3,2,2.ИСУП.5875100,11JNB13AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,125,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
4,3,2.ИСУП.5875101,11JNB21AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,150,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
5,4,2.ИСУП.5875102,12JNB41AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,250,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
6,5,2.ИСУП.5875103,12JNB33AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,300,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
7,6,2.ИСУП.5875104,11JNB13AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
8,7,2.ИСУП.5875105,12JNB33AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,8,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
9,8,2.ИСУП.5875106,12JNB41AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,4,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
10,9,2.ИСУП.5875107,11JNB21AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,1,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"


In [198]:
data1 = data[1]

data1[5:6]

6    2.ИСУП.5875103
Name: 1, dtype: object

In [186]:
data = pd.DataFrame(pd.read_excel(name_files, header=None))
data = data.dropna(thresh=5) #Удаление строк.
data = data.dropna(axis = 1, thresh= (data.shape[1] // 5)) #Удаление столбцов.
data.reset_index(inplace = True, drop = True) #Сбрасываем индекс.
n = 10
flag = False
for y in range(data.shape[0]): 
    count = 1
    for x in range(n):
        if flag:
            break
        if count >= 5:
            data = data.drop(list(data.index.values)[y])
            flag = True
            break
        if data.iloc[y, x] == count:
            count += 1
data

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,38,39,40,41,42,43
0,№ п/п,Идентификатор,Маркировка арматуры,Наименование,Тип,Класс и группа безопас-ности изделия по НП-68-05,Категория ОК,Оборудование/Материалы,"DN(арматуры), мм","Pp (арматура АЭС), Pу (общепром. арматура), МПа",...,Дата выплаты авансового платежа,Размер авансового платежа (30% от стоимости Об...,"Размер авансового платежа руб., в т.ч. НДС",Объект проектирования,Примечание,Срок поставки,Разработчик РД,Код ЕОС НСИ (GID),задание,ДЕЛЕНИЕ
2,1,2.ИСУП.5872275,11LAB80AA002,Задвижка,ЗД30-500-120-250-Э/П-Н с электроприводом Э/П,2BIIIa,QA2,оборудование\r\n,200,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA),10UJE; tсраб=150 s; Inom=20.5 A. tсраб=150 s;...,2020-03-29 00:00:00,АЭП;БКП-1;Ооб,1037211,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
3,2,2.ИСУП.5875100,11JNB13AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,125,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
4,3,2.ИСУП.5875101,11JNB21AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,150,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
5,4,2.ИСУП.5875102,12JNB41AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,250,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
6,5,2.ИСУП.5875103,12JNB33AA001,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,300,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
7,6,2.ИСУП.5875104,11JNB13AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,13.5,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
8,7,2.ИСУП.5875105,12JNB33AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,8,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
9,8,2.ИСУП.5875106,12JNB41AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,4,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"
10,9,2.ИСУП.5875107,11JNB21AA003,Задвижка,ЗД30-125-110-300-Э/О-Н с электроприводом Э/О,2BIIa,QA2,оборудование\r\n,200,1,...,2018-11-30 00:00:00,0.3,0,Реакторное здание (10UJA); код помещения 10UJ...,tсраб=30 s; Inom=10.5 A; Коэф. гидр. сопр.: 0....,2020-02-28 00:00:00,АЭП;БКП-1;Ооб,1037218,ЗЗ № 2018-105 от 01.02.2018,"Февраль К005 Задвижки ВД DN 125,DN 500"


In [49]:
result = pd.DataFrame(np.zeros((0,0 )))
result

""


In [166]:
data.columns.values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 16, 17, 18,
       19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39,
       40, 41, 42, 43], dtype=int64)